In [2]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to https://aritter.github.io/CS-7650/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)

#Project \#1: Text Classification

In this assignment, you will implement the perceptron algorithm, and a simple, but competitive neural bag-of-words model, as described in [this paper](https://www.aclweb.org/anthology/P15-1162.pdf) for text classification.  You will train your models on a (provided) dataset of positive and negative movie reviews and report accuracy on a test set.

In this notebook, we provide you with starter code to read in the data and evaluate the performance of your models.  After completing the instructions below, please follow the instructions at the end to submit your notebook and other files to Gradescope.

Make sure to make a copy of this notebook, so your changes are saved.

#Download the dataset

First you will need to download the IMDB dataset - to do this, simply run the cell below.  We have prepared a small version of the ACL IMDB dataset for you to use to help make your experiments faster.  The full dataset is available [here](https://ai.stanford.edu/~amaas/data/sentiment/), in case you are interested, but there is no need to use this for the assignment.

Note that files downloaded in Colab are only saved temporariliy - if your session reconnects you will need to re-download the files.  In case you need persistent storage, you can mount your Google drive folder like so:

```
from google.colab import drive
drive.mount('/content/drive')
```

You can also open a command line prompt by clicking on the shell icon on the left hand side of the page, and upload/download files from your local machine by clicking on the file icon.

In [3]:
#Download the data

!wget -O aclImdb_small.tgz https://github.com/aritter/CS-7650-sp22/blob/master/slides/aclImdb_small.tgz?raw=true
!tar xvzf aclImdb_small.tgz > /dev/null

--2023-02-05 23:50:10--  https://github.com/aritter/CS-7650-sp22/blob/master/slides/aclImdb_small.tgz?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/aritter/CS-7650-sp22/raw/master/slides/aclImdb_small.tgz [following]
--2023-02-05 23:50:11--  https://github.com/aritter/CS-7650-sp22/raw/master/slides/aclImdb_small.tgz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aritter/CS-7650-sp22/master/slides/aclImdb_small.tgz [following]
--2023-02-05 23:50:11--  https://raw.githubusercontent.com/aritter/CS-7650-sp22/master/slides/aclImdb_small.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.

# Converting text to numbers

Below is some code we are providing you to read in the IMDB dataset, perform tokenization (using `nltk`), and convert words into indices.  Please don't modify this code in your submitted version.  We will provide example usage below.

In [4]:
import os
import sys

import nltk
from nltk import word_tokenize
nltk.download('punkt')
import torch

#Sparse matrix implementation
from scipy.sparse import csr_matrix
import numpy as np
from collections import Counter

np.random.seed(1)

class Vocab:
    def __init__(self, vocabFile=None):
        self.locked = False
        self.nextId = 0
        self.word2id = {}
        self.id2word = {}
        if vocabFile:
            for line in open(vocabFile):
                line = line.rstrip('\n')
                (word, wid) = line.split('\t')
                self.word2id[word] = int(wid)
                self.id2word[wid] = word
                self.nextId = max(self.nextId, int(wid) + 1)

    def GetID(self, word):
        if not word in self.word2id:
            if self.locked:
                return -1        #UNK token is -1.
            else:
                self.word2id[word] = self.nextId
                self.id2word[self.word2id[word]] = word
                self.nextId += 1
        return self.word2id[word]

    def HasWord(self, word):
        return self.word2id.has_key(word)

    def HasId(self, wid):
        return self.id2word.has_key(wid)

    def GetWord(self, wid):
        return self.id2word[wid]

    def SaveVocab(self, vocabFile):
        fOut = open(vocabFile, 'w')
        for word in self.word2id.keys():
            fOut.write("%s\t%s\n" % (word, self.word2id[word]))

    def GetVocabSize(self):
        #return self.nextId-1
        return self.nextId

    def GetWords(self):
        return self.word2id.keys()

    def Lock(self):
        self.locked = True

class IMDBdata:
    def __init__(self, directory, vocab=None):
        """ Reads in data into sparse matrix format """
        pFiles = os.listdir("%s/pos" % directory)
        nFiles = os.listdir("%s/neg" % directory)

        if not vocab:
            self.vocab = Vocab()
        else:
            self.vocab = vocab

        #For csr_matrix (see http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)
        X_values = []
        X_row_indices = []
        X_col_indices = []
        Y = []

        XwordList = []
        XfileList = []

        #Read positive files
        for i in range(len(pFiles)):
            f = pFiles[i]
            for line in open("%s/pos/%s" % (directory, f)):
                wordList   = [self.vocab.GetID(w.lower()) for w in word_tokenize(line) if self.vocab.GetID(w.lower()) >= 0]
                XwordList.append(wordList)
                XfileList.append(f)
                wordCounts = Counter(wordList)
                for (wordId, count) in wordCounts.items():
                    if wordId >= 0:
                        X_row_indices.append(i)
                        X_col_indices.append(wordId)
                        X_values.append(count)
            Y.append(+1.0)

        #Read negative files
        for i in range(len(nFiles)):
            f = nFiles[i]
            for line in open("%s/neg/%s" % (directory, f)):
                wordList   = [self.vocab.GetID(w.lower()) for w in word_tokenize(line) if self.vocab.GetID(w.lower()) >= 0]
                XwordList.append(wordList)
                XfileList.append(f)
                wordCounts = Counter(wordList)
                for (wordId, count) in wordCounts.items():
                    if wordId >= 0:
                        X_row_indices.append(len(pFiles)+i)
                        X_col_indices.append(wordId)
                        X_values.append(count)
            Y.append(-1.0)
            
        self.vocab.Lock()

        #Create a sparse matrix in csr format
        self.X = csr_matrix((X_values, (X_row_indices, X_col_indices)), shape=(max(X_row_indices)+1, self.vocab.GetVocabSize()))
        self.Y = np.asarray(Y)

        #Randomly shuffle
        index = np.arange(self.X.shape[0])
        np.random.shuffle(index)
        self.X = self.X[index,:]
        self.XwordList = [torch.LongTensor(XwordList[i]) for i in index]  #Two different sparse formats, csr and lists of IDs (XwordList).
        self.XfileList = [XfileList[i] for i in index]
        self.Y = self.Y[index]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The code below reads the train,dev and test data into memory.  This will take a minute or so.

> Indented block



In [5]:
train = IMDBdata("aclImdb_small/train")
train.vocab.Lock()
dev  = IMDBdata("aclImdb_small/dev", vocab=train.vocab)
test  = IMDBdata("aclImdb_small/test", vocab=train.vocab)

# Exploring the data

Below are a few examples to help you get familiar with how the data is represented.  $X \in \mathbb{R}^{N \times M}$ contains the design matrix and $Y \in \{+1,-1\}^N$ contains the labels.  Because there are a large number of features in this representation ($X$ contains one column representing each unique word in the dataset), we are using a sparse matrix representation (Numpy's [csr_sparse](http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)).  PyTorch doesn't have great support for sparse matrices...


In [6]:
# X contains the design matrix representing the training data.
print(f"Train.X has {train.X.shape[0]} rows and {train.X.shape[1]} columns.")

Train.X has 7222 rows and 57202 columns.


In [7]:
# Labels
train.Y

array([-1., -1., -1., ..., -1.,  1., -1.])

In [8]:
# Let's count the frequency of every word appearing in the documents with negative sentiment:
word_counts = np.array(train.X[train.Y == -1,:].sum(axis=0)).flatten()
word_counts

array([ 2404, 14689,    18, ...,     1,     1,     1])

In [9]:
# Now, let's sort the words by frequency:
sorted_words = list(reversed(np.argsort(word_counts)))
sorted_words[0:10]

[16, 24, 12, 6, 25, 72, 18, 146, 143, 145]

In [10]:
# What is the index of the most frequent word?
sorted_words[0]

16

In [11]:
# Let's see what word that is:
train.vocab.GetWord(sorted_words[0])

'the'

In [12]:
# What are the 10 most frequent words?
[train.vocab.GetWord(sorted_words[x]) for x in range(10)]

['the', ',', '.', 'a', 'and', 'to', 'of', '>', '<', '/']

# Now it's time to write some code!

# Basic Perceptron (10 points)

Implement the perceptron classification algorithm (fill in the missing code marked with `TODO:` below).
The only hyperparameter to tune is the number of iterations.
Accuracy > 80% is acceptable.

In [13]:
class Eval:
    def __init__(self, pred, gold):
        self.pred = pred
        self.gold = gold
        
    def Accuracy(self):
        return np.sum(np.equal(self.pred, self.gold)) / float(len(self.gold))

class Perceptron:
    def __init__(self, X, Y, N_ITERATIONS):
        #TODO: Initalize parameters
        #X = X.toarray()
        self.N, self.M = X.shape
        self.iter = N_ITERATIONS
        #X_bias = np.concatenate((np.ones([self.N,1]),X),axis=1) # concatenate bias term
        self.w = np.zeros([self.M,1])
        self.w_avg = np.zeros([self.M,1])
        self.c = 1
        self.Train(X,Y)

    def ComputeAverageParameters(self):
        #TODO: Compute average parameters (do this part last)
        self.w -= self.w_avg/self.c

    def Train(self, X, Y):
        #TODO: Estimate perceptron parameters
        pred = np.zeros([np.shape(X)[0],1])
        for i in range(self.iter):
          for n in range(self.N):
            x = X[n].toarray()
            pred[n] = self.Predict(x)
            self.w += (Y[n] - pred[n]) * x.T
            self.w_avg += self.c * (Y[n] - pred[n]) * x.T
            self.c += 1

    def Predict(self, x):
        #TODO: Implement perceptron classification
        return [1 if i else -1 for i in (x @ self.w)>=0]

    def SavePredictions(self, data, outFile):
        Y_pred = self.Predict(data.X)
        fOut = open(outFile, 'w')
        for i in range(len(data.XfileList)):
          fOut.write(f"{data.XfileList[i]}\t{Y_pred[i]}\n")

    def Eval(self, X_test, Y_test):
        Y_pred = self.Predict(X_test)
        ev = Eval(Y_pred, Y_test)
        return ev.Accuracy()
    
ptron = Perceptron(train.X, train.Y, 10)
print(f'Q1 Basic Perceptron Acc = {ptron.Eval(dev.X, dev.Y)}')
#print(f'Q1 Basic Perceptron Acc Test = {ptron.Eval(test.X, test.Y)}')
ptron.ComputeAverageParameters()
print(f'Q2 Parameter Averaging Acc = {ptron.Eval(dev.X, dev.Y)}')
#print(f'Q2 Parameter Averaging Acc Test = {ptron.Eval(test.X, test.Y)}')

ptron.SavePredictions(test, 'test_pred_perceptron.txt')

# print(train.X.shape)
# print(test.X.shape)

Q1 Basic Perceptron Acc = 0.8465346534653465
Q2 Parameter Averaging Acc = 0.851035103510351


In [14]:
#TODO: Print the 20 most positive and 20 most negative words
sorted_words = np.argsort(ptron.w.T[0])

# 20 MOST NEGATIVE
[f'{i+1}. {train.vocab.GetWord(val)} = {ptron.w[val,0]}' for i, val in enumerate(sorted_words[0:20:1])]

['1. worst = -457.510530178203',
 '2. bad = -351.4470860276097',
 '3. poor = -310.4644909375389',
 '4. minutes = -307.71778291632626',
 '5. boring = -300.53156284183274',
 '6. instead = -298.52560889491974',
 '7. waste = -289.8265878345634',
 '8. unfortunately = -282.3615568878858',
 '9. awful = -278.39650517162596',
 '10. terrible = -266.5251658104983',
 '11. nothing = -240.113124991346',
 '12. horrible = -234.30304205148087',
 '13. dull = -223.7114689633209',
 '14. no = -218.15521801138172',
 '15. half = -215.74065714958255',
 '16. stupid = -211.93049113138838',
 '17. worse = -208.87276553910914',
 '18. annoying = -207.85253596599327',
 '19. ? = -205.16423201008018',
 '20. plot = -203.15243488735962']

In [15]:
# 20 MOST POSITIVE
[f'{i+1}. {train.vocab.GetWord(val)} = {ptron.w[val,0]}' for i, val in enumerate(sorted_words[-1:-21:-1])]

['1. great = 327.77937165090486',
 '2. excellent = 315.5986208997383',
 '3. perfect = 277.2411348499744',
 '4. fun = 226.45590617687378',
 '5. best = 218.3999667686684',
 '6. brilliant = 209.74486645158612',
 '7. especially = 208.8593622353609',
 '8. still = 201.90720150648704',
 '9. highly = 199.83636338461113',
 '10. loved = 191.56743883358024',
 '11. tarzan = 180.74629262956756',
 '12. today = 179.35929992661414',
 '13. enjoyed = 176.05017931072678',
 '14. welles = 175.4325750128079',
 '15. beautiful = 175.0361806122873',
 '16. tony = 173.5277827778624',
 '17. wonderful = 170.71040279143185',
 '18. superb = 167.54672463687848',
 '19. modesty = 166.17039365281568',
 '20. right = 164.94761911355423']

# Parameter Averaging (5 points)

Once your basic perceptron implementation is working, modify code to implement parameter averaging.  Instead of using parameters from the final iteration, $w_n$, to classify test examples, 
use the average of the parameters from every iteration, $\frac{1}{N}\sum_{i=1}^N w_i$.  A nice trick for doing this efficiently is described in section 2.1.1 of [Hal Daume's thesis](http://www.umiacs.umd.edu/~hal/docs/daume06thesis.pdf).  When you are done, save a copy of your predictions on the test set to turn in at the end (click on the file icon on the left side of the notebook).
For this part, accuracy > 82% is acceptable.

# Inspect Feature Weights (5 points)

Modify the code block above to print out the 20 most positive and 20 most negative words in the vocabulary sorted by their learned weights according to your model.  This will require a bit of thought how to do because the words in each document have been converted to IDs (see examples above).

# Evaluate on the test set

Once you are happy with your performance on the dev set, run the cell below to evaluate your performance on the test set.  Make sure to download the predictions of your model in `test_pred_perceptron.txt`.

In [16]:
print(ptron.Eval(test.X, test.Y))

ptron.SavePredictions(test, 'test_pred_perceptron.txt')

0.8507338687344226


# PyTorch Introduction

Take a look at the code below, which provides an example of how a simple neural network is capable of learning the XOR function (note that the perceptron cannot do this).  Refer to the PyTorch documentation [here](https://pytorch.org/docs/stable/index.html) for more information.

In [17]:
import torch
import torch.nn as nn
from torch import optim
import random
import numpy as np

#Define the computation graph; one layer hidden network
class FFNN(nn.Module):
    def __init__(self, dim_i, dim_h, dim_o):
        super(FFNN, self).__init__()
        self.V = nn.Linear(dim_i, dim_h)
        self.g = nn.Tanh()
        self.W = nn.Linear(dim_h, dim_o)
        self.logSoftmax = nn.LogSoftmax(dim=0)

    def forward(self, x):
        return self.logSoftmax(self.W(self.g(self.V(x))))

train_X = np.array([[0,0], [0,1], [1,0], [1,1]])
train_Y = np.array([0,1,1,0])

num_classes  = 2
num_hidden   = 10
num_features = 2

ffnn = FFNN(num_features, num_hidden, num_classes)
optimizer = optim.Adam(ffnn.parameters(), lr=0.1)

for epoch in range(100):
    total_loss = 0.0
    #Randomly shuffle examples in each epoch
    shuffled_i = list(range(0,len(train_Y)))
    random.shuffle(shuffled_i)
    for i in shuffled_i:
        x        = torch.from_numpy(train_X[i]).float()
        y_onehot = torch.zeros(num_classes)
        y_onehot[train_Y[i]] = 1

        ffnn.zero_grad()
        logProbs = ffnn.forward(x)
        loss = torch.neg(logProbs).dot(y_onehot)
        total_loss += loss
        
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:    
      print("loss on epoch %i: %f" % (epoch, total_loss))

#Evaluate on the training set:
num_errors = 0
for i in range(len(train_Y)):
    x = torch.from_numpy(train_X[i]).float()
    y = train_Y[i]
    logProbs = ffnn.forward(x)
    prediction = torch.argmax(logProbs)
    if y != prediction:
        num_errors += 1
print("number of errors: %d" % num_errors)

loss on epoch 0: 4.681751
loss on epoch 10: 2.628471
loss on epoch 20: 0.939356
loss on epoch 30: 0.144626
loss on epoch 40: 0.053400
loss on epoch 50: 0.032122
loss on epoch 60: 0.021996
loss on epoch 70: 0.016220
loss on epoch 80: 0.012585
loss on epoch 90: 0.010081
number of errors: 0


# Neural Bag-of-Words (10 points)

Your next task is to implement a neural bag-of-words baseline, NBOW-RAND, as described in [this paper](https://www.aclweb.org/anthology/P15-1162.pdf).  See section 2.1.  Note that the dataset we are using is a sample of the full IMDB dataset to make developing your solution easier, so your performance will be a bit lower than the numbers reported in the paper for the full dataset.

# GPUs

You probably want to use a GPU to enable faster training with larger word embeddings and hidden layers (the paper listed above uses 300 dimensions).

Colab provides free access to GPUs, which will be useful for rest of the assignment.  To access a GPU, select `Runtime` from the menu at the top of the page, and then choose `Change Runtime Type`; under the `Hardware Accelerator` dropdown select `GPU`.  Note that the free version of Colab does have quotas on how much GPU can be used - you won't need to use a GPU for the first part of the assignment (implementing the Perceptron algorithm).

You can check to make sure a GPU is available using the code below:

In [18]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Feb  5 23:51:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    50W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data format

In addition to reading in the data in sparse matrix (Numpy CSR) format, which you used in your implementation of the perceptron algorithm, the data loading code above also reads it in another format in `train.XwordList`.  This contains a list of PyTorch arrays consisting of sequences of word IDs corresponding to the content of each document.  The documents in `train.XwordList` are presented in the same order as labels (`train.Y`).  For the next part of the assignment, you should work with the data in this new format, which is fairly standard for text data in neural networks.  See below for a few examples on how to access the data in this new format.

In [19]:
# Example of using XwordList

print("Here is the first document:")
print(train.XwordList[0].tolist())

print("After converting IDs to words:")
print([train.vocab.GetWord(x) for x in train.XwordList[0].tolist()])

print("Here is the label:")
print(train.Y[0])

Here is the first document:
[248, 11784, 40421, 38447, 394, 52, 4, 33, 4662, 718, 18, 430, 40422, 1156, 2868, 764, 24, 331, 16, 24516, 126, 394, 69, 16, 1211, 72, 6617, 34, 237, 5699, 340, 101, 46, 498, 143, 144, 145, 146, 143, 144, 145, 146, 1188, 1, 16, 2371, 43, 246, 72, 1167, 31, 108, 99, 72, 40423, 4, 6250, 473, 12, 16, 2774, 91, 21603, 25, 40424, 12, 16, 10133, 1, 7645, 1094, 12, 12821, 498, 498, 498, 498, 498, 498, 498, 16, 2167, 1, 1701, 40425, 12, 41, 1, 6299, 72, 6, 2767, 8993, 9, 12, 143, 144, 145, 146, 143, 144, 145, 146, 248, 4, 33, 1, 260, 5150, 260, 248, 6, 156, 25, 6, 1011, 1339, 40, 5688, 374, 869, 12091, 66, 9623, 5382, 68, 12, 143, 144, 145, 146, 143, 144, 145, 146, 108, 8182, 4, 33, 25, 1692, 430, 2103, 791, 498, 498, 498]
After converting IDs to words:
['watching', 'josh', 'kornbluth', "'act", "'", 'in', 'this', 'movie', 'reminds', 'me', 'of', 'my', 'freshman', 'tv', 'production', 'class', ',', 'where', 'the', "'not", 'funny', "'", 'had', 'the', 'chance', 'to', 'pr

# Suggestions for getting started

We recommend following a similar structure as the XOR example above, starting with an [Embedding Layer](https://pytorch.org/docs/stable/nn.html\#embedding), a single hidden layer and [Adam optimizer.](https://pytorch.org/docs/stable/optim.html\#torch.optim.Adam)
Please refer to the Pytorch Documentation for more information as necessary.  You can use a batch size of one for this assignment, to simplify your implementation.
Feel free to change the number of iterations and learning rate. For this part, accuracy > 83% is acceptable.

In [20]:
import tqdm

class NBOW(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=300, NUM_CLASSES=2):
        super(NBOW, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        #TODO: Initialize parameters.
        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE,embedding_dim=DIM_EMB)
        #self.embedding_avg = nn.AvgPool1d(kernel_size=DIM_EMB)
        self.hidden = nn.Linear(in_features=DIM_EMB,out_features=NUM_CLASSES,bias=True) #nn.mean()
        self.logSoftmax = nn.LogSoftmax(dim=0)

    def forward(self, X):
        #TODO: Implement forward computation.
        #return torch.randn(self.NUM_CLASSES)
        embedding = self.embedding(X) #,torch.zeros(X.size(0),dtype=torch.long))
        #print('embedding size: ' + str(embedding.size()))
        embedding_avg = embedding.mean(dim=0)
        #print('embedding avg size: ' + str(embedding_avg.size()))
        hidden = self.hidden(embedding_avg)
        #print('hidden size: ' + str(hidden.size()))
        return self.logSoftmax(hidden)

def EvalNet(data, net):
    num_correct = 0
    Y = (data.Y + 1.0) / 2.0
    X = data.XwordList
    for i in range(len(X)):
        logProbs = net.forward(X[i])
        pred = torch.argmax(logProbs)
        if pred == Y[i]:
            num_correct += 1
    print("Accuracy: %s" % (float(num_correct) / float(len(X))))

def SavePredictions(data, outFile, net):
    fOut = open(outFile, 'w')
    for i in range(len(data.XwordList)):
        logProbs = net.forward(data.XwordList[i])
        pred = torch.argmax(logProbs)
        fOut.write(f"{data.XfileList[i]}\t{pred}\n")

def Train(net, X, Y, n_iter, dev):
    print("Start Training!")
    
    #TODO: initialize optimizer.
    optimizer = optim.Adam(net.parameters(), lr=0.1)

    num_classes = len(set(Y))

    for epoch in range(n_iter):
        num_correct = 0
        total_loss = 0.0
        net.train()   #Put the network into training mode
        for i in tqdm.notebook.tqdm(range(len(X))):
            #TODO: compute gradients, do parameter update, compute loss.
            net.zero_grad()
            #print(len(X))
            #print(X[i].size())
            y_onehot = torch.zeros(num_classes)
            y_onehot[int(Y[i])] = 1
            logProbs = net.forward(X[i])
            #print(torch.neg(logProbs).size())
            loss = torch.neg(logProbs).dot(y_onehot)
            total_loss += loss

            loss.backward()
            optimizer.step()
        net.eval()    #Switch to eval mode
        print(f"loss on epoch {epoch} = {total_loss}")
        EvalNet(dev, net)

nbow = NBOW(train.vocab.GetVocabSize())
Train(nbow, train.XwordList, (train.Y + 1.0) / 2.0, 5, dev)

Start Training!


  0%|          | 0/7222 [00:00<?, ?it/s]

loss on epoch 0 = 38976.89453125
Accuracy: 0.729072907290729


  0%|          | 0/7222 [00:00<?, ?it/s]

loss on epoch 1 = 47373.22265625
Accuracy: 0.8379837983798379


...

loss on epoch 2 = 43431.7578125
Accuracy: 0.8168316831683168


  0%|          | 0/7222 [00:00<?, ?it/s]

loss on epoch 3 = 45340.90625
Accuracy: 0.8100810081008101


  0%|          | 0/7222 [00:00<?, ?it/s]

loss on epoch 4 = 38917.22265625
Accuracy: 0.8541854185418541


# Evaluate on the test set

Once you are happy with your performance on the dev set, run the cell below to evaluate your performance on the test set.  Make sure to download the predictions of your model in `test_pred_nbow.txt`.

In [21]:
EvalNet(test, nbow)
SavePredictions(test, 'test_pred_nbow.txt', nbow)

Accuracy: 0.8414566602049294


# 1-D Convolutional Neural Networks (2 points - optional extra credit)

The final task for this assignment is to implement a convolutional neural network for text classification similar to the CNN-rand baseline described by [Kim (2014)](https://www.aclweb.org/anthology/D14-1181.pdf).  We haven't covered CNNs in lecture yet, so this part is optional.  You should use the same data format as the NBOW model above, starting out with an [Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) layer, followed by a [1-D convolution](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html), a [max-pooling layer](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html#torch.nn.MaxPool1d) and a [Dropout layer](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html).  You should be able to use the same `Train()` function you wrote above. Accuracy > 83% is acceptable.

In [ ]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, VOCAB_SIZE, DIM_EMB=300, NUM_CLASSES=2):
        super(CNN, self).__init__()
        self.NUM_CLASSES=NUM_CLASSES
        #TODO: Initialize parameters.

    def forward(self, X):
        #TODO: Implement forward computation.
        return torch.randn(self.NUM_CLASSES)

cnn = CNN(train.vocab.GetVocabSize()).cuda()
Train(cnn, train.XwordList, (train.Y + 1.0) / 2.0, 5, dev)

# Evaluate on the test set

Once you are happy with your performance on the dev set, run the cell below to evaluate your performance on the test set.  Make sure to download the predictions of your model in `test_pred_nbow.txt`.

In [ ]:
EvalNet(test, cnn)
SavePredictions(test, 'test_pred_cnn.txt', cnn)

## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_perceptron.txt
* test_pred_nbow.txt
* test_pred_cnn.txt (optional)
* TextClassification_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.

